# Default building control setting up an empty action interface

When you want to run a simulation with all the default building controllers (specified in the IDF file). You can directly use the **EnergyPlus simulation engine**. For example, placing us in the workspace of the container would be to run the following:

```bash
$ energyplus -w sinergym/data/weather/USA_PA_Pittsburgh-Allegheny.County.AP.725205_TMY3.epw sinergym/data/buildings/5ZoneAutoDXVAV.idf 
```

However, doing so without our framework has some **disadvantages**. You will have EnergyPlus default output and will not have some **added outputs** such as our wrapper logger that monitors all interactions with the environment. The IDFs have a default *location* and *designday*, which Sinergym changes automatically depending on the specified weather, so you would have to **adjust it** before using the simulator manually. Finally, the *runperiod* of the IDFs are set to one episode for DRL, which means that as the IDFs stand you can only simulate one year. So, you would also have to modify the *runperiod* **manually** in the IDF before starting the simulation.

Therefore, our recommended proposal is setting up an **empty action interface** in an environment with Sinergym. For example:

In [1]:
import gym
import numpy as np

import sinergym
from sinergym.utils.wrappers import LoggerWrapper

env = gym.make(
    'Eplus-office-hot-continuous-v1',
    action_variables=[],
    action_space=gym.spaces.Box(
        low=0,
        high=0,
        shape=(0,)),
    action_definition=None)
env = LoggerWrapper(env)

for i in range(1):
    obs = env.reset()
    rewards = []
    done = False
    current_month = 0
    while not done:
        a = env.action_space.sample()
        obs, reward, done, info = env.step(a)
        rewards.append(reward)
        if info['month'] != current_month:  # display results every month
            current_month = info['month']
            print('Reward: ', sum(rewards), info)
    print(
        'Episode ',
        i,
        'Mean reward: ',
        np.mean(rewards),
        'Cumulative reward: ',
        sum(rewards))
env.close()

/usr/local/lib/python3.10/dist-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(


[2022-12-01 12:14:14,314] EPLUS_ENV_office-hot-continuous-v1_MainThread_ROOT INFO:Updating idf ExternalInterface object if it is not present...
[2022-12-01 12:14:14,315] EPLUS_ENV_office-hot-continuous-v1_MainThread_ROOT INFO:Updating idf Site:Location and SizingPeriod:DesignDay(s) to weather and ddy file...
[2022-12-01 12:14:14,317] EPLUS_ENV_office-hot-continuous-v1_MainThread_ROOT INFO:Updating idf OutPut:Variable and variables XML tree model for BVCTB connection.
[2022-12-01 12:14:14,318] EPLUS_ENV_office-hot-continuous-v1_MainThread_ROOT INFO:Setting up extra configuration in building model if exists...
[2022-12-01 12:14:14,318] EPLUS_ENV_office-hot-continuous-v1_MainThread_ROOT INFO:Setting up action definition in building model if exists...
[2022-12-01 12:14:14,320] EPLUS_ENV_office-hot-continuous-v1_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode...
[2022-12-01 12:14:14,341] EPLUS_ENV_office-hot-continuous-v1_MainThread_ROOT INFO:EnergyPlus working directory is 

/usr/local/lib/python3.10/dist-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(


Reward:  -0.008417052432114895 {'timestep': 1, 'time_elapsed': 900, 'year': 1991, 'month': 1, 'day': 1, 'hour': 0, 'total_power': 168.3410486422979, 'total_power_no_units': -0.01683410486422979, 'comfort_penalty': -0.0, 'abs_comfort': 0.0, 'temperatures': [22.41070439945542, 19.27041626118706, 21.84918155106144, 21.88116986202595, 22.74532738560474, 21.73841316494982, 19.38460288121428, 19.66372873926693, 21.24314170888169, 20.12321836432302, 19.67730731535304, 19.88721275822578, 21.17223251340036, 20.30793764011797, 20.20143741435643, 20.4627284531045, 22.03096300421944, 20.93365898954337], 'out_temperature': 4.4, 'action_': []}
Reward:  -1482.85953514518 {'timestep': 2976, 'time_elapsed': 2678400, 'year': 1991, 'month': 2, 'day': 1, 'hour': 0, 'total_power': 249.4442211987932, 'total_power_no_units': -0.024944422119879323, 'comfort_penalty': -0.0, 'abs_comfort': 0.0, 'temperatures': [23.69669852361374, 22.25313699035975, 24.31323981198721, 23.87141679758788, 25.11090427756838, 24.562

In this example, a default environment is loaded, but the space and definition of the default action **is replaced with an empty one**, Sinergym takes care of making the necessary changes in the background. Then, the random agent implemented send **empty actions ([])**.

The advantages are that you can **combine the weathers with the buildings** as you want, Sinergym will take care of adapting everything automatically (you don't have the disadvantages of before), you can run in a single experiment as many years as you want, with our loggers. When you set an empty action interface, Sinergym preserves the default actuators that the IDF comes with (these can be more or less sophisticated depending on the definition of the building in the IDF).